In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Sickle_Cell_Anemia/GSE84634'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression of peripheral blood mononuclear cells from adults with sickle cell disease (University of Chicago cohort)"
!Series_summary	"Sickle cell disease is associated with systemic complications, many associated with either severity of disease or increased risk of mortality. We sought to identify a circulating gene expression profile whose predictive capacity spanned the spectrum of these poor outcomes in sickle cell disease."
!Series_summary	"The Training cohort consisted of patients with SCD who were prospectively recruited from the University of Illinois. The Testing cohort consisted of a combination of patients prospectively seen at two separate institutions including the University of Chicago and Howard University."
!Series_overall_design	"The gene expression of PBMC from 38 sickle cell disease patients from University of Chicago were analyzed."
Sample Characteristics Dictionary:
{0: ['tissue: peripheral blood'], 1: ['cell type: mononu

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if the dataset contains gene expression data
is_gene_available = True  # Based on series_title and series_summary indicating gene expression data

# Determine the availability of variables in the sample characteristics dictionary
sample_characteristics = {0: ['tissue: peripheral blood'], 1: ['cell type: mononuclear cells'], 2: ['disease: sickle cell disease']}
trait_row = 2 if len(set(sample_characteristics.get(2, []))) > 1 else None
age_row = None  # No age-related data in the sample characteristics
gender_row = None  # No gender-related data in the sample characteristics

# Define the conversion function
def convert_trait(value):
    if value == 'disease: sickle cell disease':
        return 1
    return None

import json

# Save cohort information
def save_cohort_info(series_id, file_path, is_gene_available, has_trait_data):
    cohort_info = {
        "series_id": series_id,
        "file_path": file_path,
        "is_gene_available": is_gene_available,
        "has_trait_data": has_trait_data
    }
    with open(file_path, "w") as file:
        json.dump(cohort_info, file)

save_cohort_info('GSE84634', './preprocessed/Sickle_Cell_Anemia/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction
if trait_row is not None:
    from utils.preprocess import geo_select_clinical_features, preview_df
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Sickle_Cell_Anemia', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Sickle_Cell_Anemia/trait_data/GSE84634.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))
